TransMed Translator is using Ginas data as a testbed since none of the groups had ingested it yet.

see:  
https://github.com/NCATS-Tangerine/ncats-ingest/issues/1

The best source for the data to date is found at:  
https://tripod.nih.gov/ginas/#/gsrs/data

 
this  notebook does assume `jq` is installed  
https://stedolan.github.io/jq/

In [37]:
# commented out after first fetch 
# wget -q --timestamping https://tripod.nih.gov/ginas/gsrs/fullSeedData-2016-06-16.gsrs

In [38]:
ls -lh *.gsrs

-rw-r--r-- 1 tomc tomc 91M Aug 12 12:37 fullSeedData-2016-06-16.gsrs


In [39]:
file fullSeedData-2016-06-16.gsrs

fullSeedData-2016-06-16.gsrs: gzip compressed data, last modified: Thu Jun 16 14:53:25 2016, from Unix


In [40]:
gunzip - < fullSeedData-2016-06-16.gsrs > fullSeedData-2016-06-16

In [41]:
file fullSeedData-2016-06-16

fullSeedData-2016-06-16: ASCII text, with very long lines


make sure it is not all on one half gigabyte line before taking a look (happens).

In [42]:
wc -l < fullSeedData-2016-06-16

84979


In [43]:
head -2 fullSeedData-2016-06-16

WORKED: 18MXK3D6DB	b029367a-c360-4617-8a2e-fd90c7a88fdf	{"structure":{"references":["17162b41-9aa8-4549-b423-a1169d864351","383105ca-e739-4df4-9be3-cb7342556d73"],"molfile":"\n  Symyx   06151614132D 1   1.00000     0.00000     0\n\n 12 12  0     0  0            999 V2000\n   21.4377   -8.1424    0.0000 C   0  0  0  0  0  0           0  0  0\n   22.4606   -8.7330    0.0000 C   0  0  0  0  0  0           0  0  0\n   23.4835   -8.1424    0.0000 C   0  0  0  0  0  0           0  0  0\n   23.4835   -6.9613    0.0000 C   0  0  0  0  0  0           0  0  0\n   22.4606   -9.9141    0.0000 C   0  0  0  0  0  0           0  0  0\n   21.4377  -10.5046    0.0000 C   0  0  0  0  0  0           0  0  0\n   21.4377  -11.6857    0.0000 S   0  0  3  0  0  0           0  0  0\n   22.6188  -11.6857    0.0000 O   0  0  0  0  0  0           0  0  0\n   20.2566  -11.6857    0.0000 O   0  0  0  0  0  0           0  0  0\n   21.4377  -12.8668    0.0000 O   0  0  0  0  0  0           0  0  0\n   20.4149   -9.9

looks like an array of:   

    WORKED: <SRS_UNII>	<UUID> {json: blob}
    

 __ASIDE__  
Where `SRS_UNII` is FDA's "Substance Registration System Unique Ingrediant Identifier"  
The second one for example, goes to:   
    https://fdasis.nlm.nih.gov/srs/auto/startswith/o061w4c37h

(Will also note that although Ginas does not have an "InChIKey" for this substance the fda does associate the inchikey "DHAZIUXMHRHVMP-UHFFFAOYSA-N"

FDA has https://fdasis.nlm.nih.gov/srs/download/srs/UNIIs.zip

with 601k rows of:  

    head -1 "UNIIs 10Nov2016 Names.txt"
    NAME	TYPE	UNII	PT
    
and https://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip

with 83k rows of:  

    head -1 "UNIIs 10Nov2016 Records.txt"
    UNII	PT	RN	EC	NCIT	RXCUI	ITIS	NCBI	PLANTS	GRIN	INN_ID	MF	INCHIKEY	SMILES	UNII_TYPE

along with 8k rows in "UNIIs 10Nov2016 Changes.txt" with:  
    
    UNII	DATA	FIELD	TYPE
    

Seems the UNIIs Records file could be a resource for filling in gaps.  
There are 48,341 unii->inchikey mappings and 35,385 unii missing mappings.

Although this file contains json it is not quite valid in its current form.

In [46]:
jq . fullSeedData-2016-06-16

parse error: Invalid numeric literal at line 1, column 7


: 4

Omitting the leading items or changing them to json are the options. Omitting is risky because although the few I have looked at the UNII and UUID are replicated in the json I can't be sure it is consistant so including them is safer.  

In [47]:
awk 'BEGIN{print "["}{$1="\"unid\": ";$2="\"" $2"\",";$3="\"ginas\": \""$3"\",";$4="\"record\": "$4;print "{" $0 "},"}END{print "]"}' fullSeedData-2016-06-16 > fullSeedData-2016-06-16.json


In [48]:
jq . fullSeedData-2016-06-16.json

parse error: Invalid numeric literal at line 33442, column 96


: 4

That is anoying. Malformed json deep in the file.

In [49]:
sed -n '33442'p fullSeedData-2016-06-16.json | cut -c1-200

{"unid":  "[ingredient]", "ginas": "3786568514", "record": 90061b24-2d13-4a1b-bfbe-46b22cf2cea2 {"substanceClass":"concept","names":[{"name":"HYDROCODONE POLISTIREX","stdName":"HYDROCODONE POLISTIREX"


Everything is displaced to the right by one with the appearence of `"[ingredient]"`.

In [50]:
grep -c "^WORKED: \[ingredient\]" fullSeedData-2016-06-16

4933


In [51]:
grep -c "^WORKED: \[" fullSeedData-2016-06-16

4933


Appears the 5k with `"[ingredient]"` are the only ones with the optional flag in that location.  


In [52]:
awk 'BEGIN{print "["}\
NR==1             {$1="\"unid\":";$2="\"" $2 "\",";$3="\"ginas\": \"" $3 "\",";$4="\"record\":" $4;print "{"$0"}"}\
NR>1&&\
$2!="[ingredient]"{$1="\"unid\":";$2="\"" $2 "\",";$3="\"ginas\": \"" $3 "\",";$4="\"record\":" $4;print ",{"$0"}"}\
$2=="[ingredient]"{$1="\"unid\":";$3="\"" $3 "\",";$4="\"ginas\": \"" $4 "\",";$5="\"ingredient\":\"true\",\"record\":" $5;$2="";print ",{"$0"}"}\
END{print "]"}' fullSeedData-2016-06-16 > fullSeedData-2016-06-16.json

In [53]:
jq . fullSeedData-2016-06-16.json > fullSeedData-2016-06-16_pp.json

In [54]:
head fullSeedData-2016-06-16_pp.json

[
  {
    "unid": "18MXK3D6DB",
    "ginas": "b029367a-c360-4617-8a2e-fd90c7a88fdf",
    "record": {
      "structure": {
        "references": [
          "17162b41-9aa8-4549-b423-a1169d864351",
          "383105ca-e739-4df4-9be3-cb7342556d73"
        ],


How are records spaced out after formatting?

In [55]:
grep -n "^  {" fullSeedData-2016-06-16_pp.json| head

2:  {
146:  {
355:  {
603:  {
822:  {
1200:  {
1626:  {
1905:  {
2199:  {
2467:  {
grep: write error: Broken pipe


Looks like a couple of hundred lines each.
What does a record look like?

In [1]:
sed -n '2199,2467'p fullSeedData-2016-06-16_pp.json

  {
    "unid": "0HBU04R8B0",
    "ginas": "9aa0b0d4-d546-47a1-8c0f-207d983b75d5",
    "record": {
      "structure": {
        "references": [
          "310a4998-172a-400b-b469-bd60962a6aa1",
          "a1ecee58-f238-423c-a3f3-cf58a3e48501"
        ],
        "molfile": "\n Symyx 06151614132D 1 1.00000 0.00000 0\n\n 14 14 0 0 0 999 V2000\n 0.1458 -4.2583 0.0000 C 0 0 0 0 0 0 0 0 0\n -1.0125 -4.9292 0.0000 N 0 0 0 0 0 0 0 0 0\n 0.1542 -2.9458 0.0000 O 0 0 0 0 0 0 0 0 0\n 1.3000 -4.9583 0.0000 O 0 0 0 0 0 0 0 0 0\n -2.1500 -4.2750 0.0000 C 0 0 0 0 0 0 0 0 0\n -3.3125 -4.9458 0.0000 C 0 0 0 0 0 0 0 0 0\n -4.4667 -4.2417 0.0000 C 0 0 0 0 0 0 0 0 0\n -5.5958 -4.9042 0.0000 Cl 0 0 0 0 0 0 0 0 0\n -3.3083 -2.2542 0.0000 C 0 0 0 0 0 0 0 0 0\n 2.4333 -4.2917 0.0000 C 0 0 3 0 0 0 0 0 0\n -2.1542 -2.9500 0.0000 C 0 0 0 0 0 0 0 0 0\n -4.4708 -2.9167 0.0000 C 0 0 0 0 0 0 0 0 0\n 3.5875 -4.9500 0.0000 C 0 0 0 0 0 0 0 0 0\n 2.4667 -2.9667 0.0000 C 0 0 0 0 0 0 0 0 0\n 2 1 1 0 0 0\n 3 1 2 0 0 0\n 4 1

parsed into ntriples with python. see the `Parse_Ginas_json.py` jupyter notebook.
and `Ginas.py`

I have not included any Moiety fields so far.  

##############################################################

In [1]:
wc -l < ginas.nt

3522185


RDF ntriples have lines of the form "`<subject> <predicate> <object> .`"  
we should be able to confirm certain conditions hold:  
\>    The subject is always a Ginias identifier.  
\>    The predicate is now always a path to root (to be reinterperted as an ontological term).  
\>    The object is most variable but when it is a Ginas identifier. That identifier exists as a subject.  

In [3]:
grep -c "^<GINAS:" ginas.nt
grep -c "^<GINASREF:" ginas.nt

1563311
1958874


The output is free of duplicate statements.  

What predicates are there and how many of each?

In [5]:
cut -f2 -d' ' ginas.nt| sort | uniq -c | sort -nr

 649506 <reference_uuid:>
 649506 <reference_docType:>
 649506 <reference_citation:>
 553496 <reference_tag:>
 205757 <name_references:>
 198471 <codes_code:>
 113086 <structure_references:>
 106366 <reference_url:>
  84979 <unii:>
  84979 <substanceClass:>
  84977 <names_stdName:>
  70718 <code_references:>
  56543 <structure_formula:>
   7437 <mixture_component_substance:>
   4933 <ingredient:>
   1925 <mixture_component_type:>


the predicates are a path within the JSON `record` object  

(except `ingredient` and `unii` which are technicaly sibblings of `record`)


Those need mapping to their proper ontology terms  

To be okay, unique object ids must not out number unique subject ids

In [12]:
cut -f1 -d' ' ginas.nt| grep "^<GINAS:" |sort -u > ginas_subject.list
cut -f1 -d' ' ginas.nt| grep "^<GINASREF:" |sort -u > ginasref_subject.list

In [13]:
wc -l < ginas_subject.list
wc -l < ginasref_subject.list

84979
649506


In [14]:
cut -f3 -d' ' ginas.nt |grep "^<GINAS:" | sort -u > ginas_object.list
cut -f3 -d' ' ginas.nt |grep "^<GINASREF:" | sort -u > ginasref_object.list

In [15]:
wc -l < ginas_object.list
wc -l < ginasref_object.list

6858
649506


Okay so far

In [16]:
comm -13 ginas_subject.list ginas_object.list

No output means all objects exist as subjects which is good.  

In [18]:
comm -13 ginasref_subject.list ginasref_object.list